In [103]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.image as mpimg
import pandas as pd
import math
import sys
from collections import defaultdict

import tensorflow as tf

In [104]:
(x_train,y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])

y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

no_classes = 10
no_features = len(x_train[0])

In [105]:
x = defaultdict(list)
X = []
mu = []
MU = []

for i in range(0,len(x_train)):
    if int(y_train[i]) < no_classes:
        x[int(y_train[i])].append(x_train[i])
        X.append(x_train[i])
        
X = np.array(X)

for i in range(0,no_features):
    MU.append(np.mean(X[:,i]))

for i in range(0,no_classes):
    tempMu = []
    tempClass = np.array(x[i])
    for j in range(0,no_features):
        tempMu.append(np.mean(tempClass[:,j]))
    mu.append(tempMu)

In [106]:
xo = x

# for i in range(0,no_classes):
#     for j in range(0,no_features):
#         for k in range(0,len(xo[i])):
#             xo[i][k][j] -= MU[j]

c = []
detc = []
invc = []

for i in range(0,no_classes):
    tempClass = np.array(xo[i])
    tempCov = np.cov(np.transpose(tempClass))
    c.append(tempCov)
    detc.append(np.linalg.det(tempCov))
    invc.append(np.linalg.pinv(tempCov))
    
C = [[0 for x in range(no_features)] for y in range(no_features)] 

for i in range(0,no_features):
    for j in range(0,no_features):
        temp = 0
        for k in range(0,no_classes):
            temp += (len(x[k])*c[k][i][j])
        C[i][j] = temp/len(X)
        
invC = np.linalg.pinv(C)
        
p = []
for i in range(0,no_classes):
    p.append(len(x[i])/len(X))

In [107]:
error = 0
total = 0

for i in range(0,len(x_test)):
    if int(y_test[i]) < no_classes:
        total += 1
        curF = -sys.maxsize
        ye = 0
        for j in range(0,no_classes):
            tempF = (np.matmul(np.matmul(np.transpose(x_test[i]),invC),mu[j]))+((-0.5)*(np.matmul(np.matmul(np.transpose(mu[j]),invC),mu[j])))+np.log(p[j])
            if tempF > curF:
                curF = tempF
                ye = j
        if ye!=int(y_test[i]):
            error += 1

print("Error(LDA) :", (error*100)/total)
print("Accuracy(LDA) :", ((total-error)*100)/total)

print("----------------------------------------")

error = 0
total = 0
for i in range(0,len(x_test)):
    if int(y_test[i]) < no_classes:
        total += 1
        curF = -sys.maxsize
        ye = 0
        for j in range(0,no_classes):
            tempF = ((-0.5)*(detc[j]+np.matmul(np.matmul(np.transpose(x_test[i]-mu[j]),invc[j]),x_test[i]-mu[j])))+np.log(p[j])
            if tempF > curF:
                curF = tempF
                ye = j
        if ye!=int(y_test[i]):
            error += 1

print("Error(QDA) :", (error*100)/total)
print("Accuracy(QDA) :", ((total-error)*100)/total)

Error(LDA) : 12.7
Accuracy(LDA) : 87.3
----------------------------------------
Error(QDA) : 14.28
Accuracy(QDA) : 85.72
